<a target="_blank" href="https://colab.research.google.com/github/josephassiga/tensorflow-and-keras-tutorial/blob/main/W4/Lab5/W4_Lab_1_intro_timeseries.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Deep Learning for timeseries